<a href="https://colab.research.google.com/github/Gops03/TEORIA_DE_SE-ALES/blob/main/redlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LSTM, Dense, Dropout  # Importar LSTM

# Parámetros de la señal
fs = 1000  # Frecuencia de muestreo
t = np.linspace(0, 3, fs * 3, endpoint=False)  # Vector de tiempo ajustado para 3 segundos

# Parámetros de las ondas senoidales
f1, f2, f3 = 100, 200, 400
A1, A2, A3 = 0.9, 0.6, 0.3
wo = 2*np.pi

# Crear la señal base x(t)
signal = A1 * np.cos(wo * f1 * t) + A2 * np.sin(wo * f2 * t) + A3 * np.cos(wo * f3 * t)

# Generar un conjunto de datos sintéticos con diferentes niveles de ruido
num_samples = 1000
X = []
y = []

for _ in range(num_samples):
    # Añadir ruido blanco gaussiano con un nivel aleatorio
    noise_level = np.random.uniform(0.0001, 0.5)  # Diferente nivel de ruido para cada muestra
    noise = np.random.normal(0, noise_level, signal.shape)
    noisy_signal = signal + noise

    # Almacenar la señal ruidosa
    X.append(noisy_signal)

    # Calcular la PSD usando Welch
    f, Pxx = welch(noisy_signal, fs, nperseg=256)
    y.append(Pxx)

# Convertir las listas en arrays numpy
X = np.array(X)
y = np.array(y)

# Verificar la forma de X para asegurar que tiene 3 dimensiones
print("Forma original de X:", X.shape)

# Si X tiene solo 2 dimensiones (samples, timesteps), necesitamos agregar una dimensión para features
if len(X.shape) == 2:
    X = X.reshape((X.shape[0], X.shape[1], 1))

print("Nueva forma de X (después de reshape):", X.shape)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir la métrica R² personalizada
def r2_keras(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res / (SS_tot + K.epsilon()))

# Crear el modelo LSTM
model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.4),
    LSTM(64, activation='relu', return_sequences=False),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l2(0.002)),
    Dense(y_train.shape[1], activation='linear')  # Salida con la misma dimensión que la PSD
])

# Compilar el modelo con un learning rate ajustado
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=[r2_keras])

# Entrenar el modelo con early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1, callbacks=[early_stopping])

# Graficar la evolución de la pérdida y el R² durante el entrenamiento
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Curva de pérdida')
plt.plot(history.history['val_loss'], label='Curva de validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Evolución de la Curva de pérdida ')

plt.subplot(1, 2, 2)
plt.plot(history.history['r2_keras'], label='Curva de entrenamiento')
plt.plot(history.history['val_r2_keras'], label='Curva de validación')
plt.xlabel('Épocas')
plt.ylabel('R²')
plt.yscale('log')  # Escala logarítmica para R²
plt.legend()
plt.title('Evolución de la Curva de entrenamiento (Escala Logarítmica)')

plt.tight_layout()
plt.show()

# Volver a predecir y comparar la PSD
y_pred = model.predict(X_test)


Forma original de X: (1000, 3000)
Nueva forma de X (después de reshape): (1000, 3000, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 949ms/step - loss: 0.1681 - r2_keras: -2.1484 - val_loss: 0.1627 - val_r2_keras: 0.1016
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 690ms/step - loss: 0.1610 - r2_keras: -0.0228 - val_loss: 0.1559 - val_r2_keras: 0.3395
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 678ms/step - loss: 0.1543 - r2_keras: 0.3473 - val_loss: 0.1493 - val_r2_keras: 0.5626
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 679ms/step - loss: 0.1478 - r2_keras: 0.5620 - val_loss: 0.1430 - val_r2_keras: 0.7238
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 696ms/step - loss: 0.1415 - r2_keras: 0.7175 - val_loss: 0.1370 - val_r2_keras: 0.8329
Epoch 6/50
 9/25 ━━━━━━━━━━━━━━━━━━━━ 9s 616ms/step - loss: 0.1365 - r2_keras: 0.8063 

KeyboardInterrupt: 

In [ ]:
# Calcular la PSD original para una señal de prueba
f_original, Pxx_original = welch(X_test[0].flatten(), fs, nperseg=256)

# Verificar las dimensiones
print(f"Dimensiones de f_original: {f_original.shape}")
print(f"Dimensiones de Pxx_original: {Pxx_original.shape}")
print(f"Dimensiones de y_pred[0]: {y_pred[0].shape}")

# Asegurarse de que las dimensiones coincidan
if f_original.shape[0] != Pxx_original.shape[0]:
    raise ValueError("Las dimensiones de f_original y Pxx_original no coinciden.")

if f_original.shape[0] != y_pred[0].shape[0]:
    raise ValueError("Las dimensiones de f_original y y_pred[0] no coinciden.")

# Graficar la PSD original y la PSD predicha para la primera señal
plt.figure(figsize=(10, 6))
plt.semilogy(f_original, Pxx_original, label='PSD Original')
plt.semilogy(f_original, y_pred[0], label='PSD Predicha')
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Densidad Espectral de Potencia (PSD)')
plt.title('Comparación de la PSD Original y Predicha')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
import numpy as np
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt

# Realizar la FFT sobre la señal original
fft_signal = fft(X_test[0].flatten())

# Tomar la magnitud y fase original
magnitude_original = np.abs(fft_signal)
phase_original = np.angle(fft_signal)

# Asegurarse de que la PSD predicha no tenga valores negativos
y_pred_clipped = np.clip(y_pred[0], a_min=0, a_max=None)

# Utilizar la PSD predicha para obtener la magnitud predicha
magnitude_pred = np.sqrt(y_pred_clipped * fs)

# Expandir la magnitud predicha para cubrir todo el espectro
# Rellenar con ceros en la frecuencia media y las negativas
magnitude_pred_full = np.zeros_like(magnitude_original)
half_length = len(magnitude_pred)
magnitude_pred_full[:half_length] = magnitude_pred  # Primeros N/2 valores
magnitude_pred_full[-half_length+1:] = magnitude_pred[1:][::-1]  # Espejo de la parte positiva

# Reconstruir la señal usando la magnitud predicha y la fase original
fft_reconstructed = magnitude_pred_full * np.exp(1j * phase_original)
reconstructed_signal = np.real(ifft(fft_reconstructed))

# Seleccionar solo una parte de la señal para graficar
start, end = 0, 1000  # Puedes ajustar estos valores según lo que quieras visualizar

# Crear los gráficos en subplots separados
plt.figure(figsize=(12, 6))

# Gráfico de la señal original
plt.subplot(2, 1, 1)
plt.plot(t[start:end], X_test[0][start:end], label='Señal Original', color='blue')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal Original')
plt.grid(True)

# Gráfico de la señal reconstruida
plt.subplot(2, 1, 2)
plt.plot(t[start:end], reconstructed_signal[start:end], label='Señal Reconstruida', color='orange')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal Reconstruida')
plt.grid(True)

plt.tight_layout()
plt.show()
